In [ ]:
import osiris_utils as ou
import numpy as np
from pathlib import Path
from tqdm import tqdm
import h5py
import sys
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import RegularGridInterpolator
from tqdm import tqdm

import contextlib
import io

plt.rcParams['font.size'] = 14

In [ ]:

sim_Boris_0_01 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/Boris/Boris_0_01/Boris_0_01.in")

sim_Boris_0_1 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/Boris/Boris_0_1/Boris_0_1.in")

sim_Boris_1 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/Boris/Boris_1/Boris_1.in")

sim_Boris_10 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/Boris/Boris_10/Boris_10.in")


sim_GcaCorr_1 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/GcaCorr/GcaCorr_1/GcaCorr_1.in")

sim_GcaCorr_10 = ou.Simulation("/home/exxxx5/Tese/Decks/ComparePushers/GcaCorr/GcaCorr_10/GcaCorr_10.in")

In [ ]:
def call_silently(func, *args, **kwargs):
    with contextlib.redirect_stdout(io.StringIO()):
        return func(*args, **kwargs)


def get_trajectory(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["x1"][particle, :][mask],
        track.data["x2"][particle, :][mask],
        track.data["x3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj

def get_p(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["px"][particle, :][mask],
        track.data["py"][particle, :][mask],
        track.data["pz"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj


def get_fields(sim, particle, tmin = 0, tmax = 100000000000, unload=False):
    track = sim["test_electrons"]["tracks"]
    track.load_all()

    mask = (track.data["t"][particle, :] >= tmin) & (track.data["t"][particle, :] <= tmax)
    traj = np.stack([
        track.data["E1"][particle, :][mask],
        track.data["E2"][particle, :][mask],
        track.data["E3"][particle, :][mask],
        track.data["B1"][particle, :][mask],
        track.data["B2"][particle, :][mask],
        track.data["B3"][particle, :][mask],
        track.data["t"][particle, :][mask]
        ], axis=0)

    if unload:
        track.unload()

    return traj

def get_gradB(sim, t=None):
    
    B = (sim["part_b1"]**2 + sim["part_b2"]**2 + sim["part_b3"]**2)**(0.5)

    dB_x1 = ou.Derivative_Diagnostic(B, "x1")
    dB_x2 = ou.Derivative_Diagnostic(B, "x2")
    dB_x3 = ou.Derivative_Diagnostic(B, "x3")

    if isinstance(t, int):
        return np.stack([
            dB_x1[t],
            dB_x2[t],
            dB_x3[t]
        ], axis=0)
    else:
        return np.stack([
            dB_x1,
            dB_x2,
            dB_x3
        ], axis=0)

def diag_interpolator(diag, t=0):
    # TODO maybe considder periodic boundaries
    
    # Define grid points
    x = np.linspace(diag.grid[0][0] + diag.dx[0], diag.grid[0][1], diag.nx[0])
    y = np.linspace(diag.grid[1][0] + diag.dx[1], diag.grid[1][1], diag.nx[1])
    z = np.linspace(diag.grid[2][0] + diag.dx[2], diag.grid[2][1], diag.nx[2])
    
    return RegularGridInterpolator((x, y, z), diag[t], bounds_error=False, fill_value=None, method='cubic')



def plot_drifts(sim, particles, tmin = 0, tmax = 100000000000, unload=False, fig=None, ax=None):
    gradB=get_gradB(sim)
    interp_gradB_1 = diag_interpolator(gradB[0], t=0)
    interp_gradB_2 = diag_interpolator(gradB[1], t=0)
    interp_gradB_3 = diag_interpolator(gradB[2], t=0)

    v_EXB=[]
    v_GradB=[]
    for particle in tqdm(particles, desc="Calculating Drifts"):
        fields = call_silently(get_fields, sim, particle, tmin, tmax)
        traj = call_silently(get_trajectory, sim, particle, tmin, tmax)
        
        E = fields[0:3, :]
        B = fields[3:6, :]
        b = B / np.linalg.norm(B, axis=0)
        t = fields[-1, :]


        # E x B drift
        v_EXB.append(np.linalg.norm(np.cross(E.T, b.T), axis=1))

        
        # GradB drift
        gradB_traj = np.stack([
            interp_gradB_1(traj[0:3].T),
            interp_gradB_2(traj[0:3].T),
            interp_gradB_3(traj[0:3].T)
        ], axis=0)

        v_GradB.append(np.linalg.norm(np.cross(b.T, gradB_traj.T), axis=1))

    if unload:
        sim["test_electrons"]["tracks"].unload()
        
    #average for every particle
    avrg_v_EXB = np.mean(np.array(v_EXB), axis=0)
    avrg_v_gradB = np.mean(np.array(v_GradB), axis=0)

    if fig is None:
        fig = plt.figure(figsize=(10, 7))

    if ax is None:
        ax = fig.add_subplot(111)

    ax.plot(t, avrg_v_EXB, label=r"$v_{\mathbf{E} \times \mathbf{B}}$", marker='o', linestyle='-', markersize = 1.8, linewidth=1)
    ax.plot(t, avrg_v_gradB, label=r"$\gamma v_{\nabla \mathbf{B}} / \mu $", marker='o', linestyle='-', markersize = 1.8, linewidth=1)

    # Labels and title
    ax.set_xlabel('${}$'.format(sim["test_electrons"]["tracks"].labels["t"]) + "$[{}]$".format(sim["test_electrons"]["tracks"].units["t"]))
    #ax.set_ylabel('mean dist to baseline' + "$[{}]$".format(sim["test_electrons"]["tracks"].units["x1"]))
    ax.legend()

    return fig, ax

In [ ]:
# Boris pusher trajectories
particle = range(0, 1000)
#particle = [400]
tmax = 700

fig, ax = None, None                            
fig, ax = plot_drifts(sim_Boris_0_1, particles=particle, tmax=tmax, fig=fig, ax=ax)

ax.set_yscale('log')
ax.set_title("Particles " + str(particle))
plt.tight_layout()
plt.show()